In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
test = torch.randn([3, 8, 60])

In [69]:
conv_1 = nn.Conv2d(3, 64, (1, 3), padding=0)
conv_2 = nn.Conv2d(64, 32, (1, 7), padding=0)
conv_3 = nn.Conv2d(32, 16, (1, 10), padding=0)
conv_4 = nn.Conv2d(16, 3, (1, 14), padding=0)

In [72]:
test = conv_4(conv_3(conv_2(conv_1(test))))

In [71]:
convt_1 = nn.ConvTranspose2d(3, 16, (1, 14), padding=0)
convt_2 = nn.ConvTranspose2d(16, 32, (1, 10), padding=0)
convt_3 = nn.ConvTranspose2d(32, 64, (1, 7), padding=0)
convt_4 = nn.ConvTranspose2d(64, 3, (1, 3), padding=0)

In [75]:
convt_4(convt_3(convt_2(convt_1(test)))).shape

torch.Size([3, 8, 60])

In [43]:
pool_1 = nn.MaxPool2d((1, 2), (1, 2))
pool_1(conv_1(test)).shape

torch.Size([16, 10, 30])

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/Close.csv', index_col=0, parse_dates=True)

In [3]:
np.array((np.log(df) - np.log(df.shift(1))).dropna()).shape

(814, 8)

In [4]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X = list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x = sequence[i:end_ix]
		X.append(seq_x)
	return np.array(X)

In [22]:
test = split_sequence(np.array((np.log(df) - np.log(df.shift(1))).dropna()), 60)

In [15]:
test.shape

(754, 60, 8)

In [6]:
a = np.expand_dims(np.transpose(test, (0, 2, 1)), axis=1)
b = np.expand_dims(np.transpose(test, (0, 2, 1)), axis=1)

In [12]:
c = np.concatenate([a, b], axis=1)

In [17]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [10]:
scaler = MinMaxScaler(feature_range=(-1, 1))

In [20]:
for i in range(test.shape[0]):
    scaler = StandardScaler()
    test[i, :, :] = scaler.fit_transform(test[i, :, :]) 

In [23]:
test

array([[[-7.34918596e-03, -2.87277383e-03, -1.46036550e-03, ...,
         -1.28618272e-03,  4.62901264e-03, -1.76423049e-03],
        [-8.35545832e-03, -3.72118520e-03, -3.21757156e-02, ...,
         -3.28278129e-02,  2.73737876e-03, -3.23662102e-02],
        [-1.67667340e-01, -5.90079885e-02, -2.22722651e-02, ...,
         -2.12731054e-02,  1.58559969e-04, -2.15537694e-02],
        ...,
        [ 1.45717371e-01,  4.30708228e-02,  1.03979094e-03, ...,
          7.82917896e-04,  1.58056160e-03,  7.83323601e-04],
        [-3.85483340e-02, -2.54692506e-02, -2.41520683e-02, ...,
         -2.41600024e-02, -4.52418154e-03, -2.34466639e-02],
        [ 3.86950338e-02,  8.74587229e-03,  3.29470389e-02, ...,
          3.24735996e-02, -5.35876033e-03,  3.25647533e-02]],

       [[-8.35545832e-03, -3.72118520e-03, -3.21757156e-02, ...,
         -3.28278129e-02,  2.73737876e-03, -3.23662102e-02],
        [-1.67667340e-01, -5.90079885e-02, -2.22722651e-02, ...,
         -2.12731054e-02,  1.58559969e

In [24]:
test.shape

(754, 60, 8)

In [27]:
from environment.env import TradingEnv

In [28]:
env = TradingEnv('data')

g:\Code\Python\GitHub\Portfolio-Optimizer\environment\env.py:42: RuntimeWarning: divide by zero encountered in true_divide
  self.gain = np.hstack((np.ones((self.close_prices.shape[0]-1, 1)), self.close_prices.values[1:] / self.close_prices.values[:-1]))
g:\Code\Python\GitHub\Portfolio-Optimizer\environment\env.py:220: DeprecationWarning: This function is deprecated. Please call randint(59, 612 + 1) instead
  self.dates_num-self.steps-1)


In [30]:
env.observation_space

Box(-inf, inf, (1, 8, 60), float32)